In [1]:
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import math

# --- 1. Mediapipe Setup ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2
)
mp_drawing = mp.solutions.drawing_utils

# --- 2. Distance Calculation Function ---
def calculate_distance(lm1, lm2, frame_width, frame_height):
    """Calculates the Euclidean distance between two normalized landmarks."""
    # Convert normalized coordinates (0 to 1) to pixel coordinates
    x1, y1 = int(lm1.x * frame_width), int(lm1.y * frame_height)
    x2, y2 = int(lm2.x * frame_width), int(lm2.y * frame_height)
    
    # Calculate Euclidean distance
    distance = math.hypot(x2 - x1, y2 - y1)
    
    return distance, (x1, y1), (x2, y2)

# --- 3. Tkinter Application Class ---
class HandDistanceApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        
        # Initialize video capture
        self.cap = cv2.VideoCapture(0)
        
        # Determine video frame size (adjust as needed)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Setup GUI elements
        self.setup_gui()
        
        # Start the update loop
        self.delay = 15 # Delay in ms, roughly 66 FPS
        self.update()

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.window.mainloop()

    def setup_gui(self):
        # Video Frame
        self.video_label = ttk.Label(self.window)
        self.video_label.pack(padx=10, pady=10)
        
        # Distance Display Label
        self.distance_var = tk.StringVar(self.window, value="Hand Distance: N/A")
        self.distance_label = ttk.Label(
            self.window, 
            textvariable=self.distance_var,
            font=('Arial', 16, 'bold')
        )
        self.distance_label.pack(padx=10, pady=5)
        
        # Instructions/Status
        ttk.Label(self.window, text="Move your Thumb and Index Finger close/far apart!").pack(pady=5)

    def update(self):
        """Main loop that runs every 'delay' ms to grab and process a video frame."""
        success, img = self.cap.read()

        if success:
            # Mirror and color convert (from original code)
            img = cv2.flip(img, 1)
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(imgRGB)
            
            current_distance_text = "Hand Distance: Not Detected"
            
            if results.multi_hand_landmarks:
                for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    
                    # --- CORE MODIFICATION: DISTANCE DETECTION ---
                    # MediaPipe Landmark IDs:
                    # THUMB_TIP = 4
                    # INDEX_FINGER_TIP = 8
                    
                    # Ensure we have the necessary landmarks
                    lm_thumb = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    lm_index = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    
                    # Calculate distance and get pixel coordinates for drawing
                    distance_px, (x1, y1), (x2, y2) = calculate_distance(
                        lm_thumb, lm_index, self.width, self.height
                    )
                    
                    # Draw connection line and circle (Optional visual feedback)
                    cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.circle(img, (x1, y1), 8, (255, 0, 255), cv2.FILLED)
                    cv2.circle(img, (x2, y2), 8, (255, 0, 255), cv2.FILLED)
                    
                    # Display distance on video frame
                    mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
                    cv2.putText(img, f'{distance_px:.2f} px', (mid_x - 50, mid_y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    
                    # Update Tkinter variable
                    current_distance_text = f"Hand Distance: {distance_px:.2f} pixels"
                    # --- END CORE MODIFICATION ---
                    
                    # Optionally draw the full hand landmarks (from original code)
                    mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                self.distance_var.set(current_distance_text)
            
            else:
                self.distance_var.set("Hand Distance: No Hand Detected")

            # Convert OpenCV image to PhotoImage for Tkinter
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
            self.video_label.config(image=self.photo)

        # Rerun the update function after 'delay' milliseconds
        self.window.after(self.delay, self.update)
        
    def on_closing(self):
        """Clean up resources before closing the application."""
        print("Releasing camera and closing application...")
        self.cap.release()
        self.window.destroy()

# --- 4. Main Execution ---
if __name__ == "__main__":
    # Ensure you are running this in the environment where cv2 and mediapipe are installed!
    root = tk.Tk()
    app = HandDistanceApp(root, "MediaPipe Hand Distance Detector")

Releasing camera and closing application...
